In [19]:
import simpy
import random

totalTime = 0 # Stores total waiting time in the process
avgSum = 0 # Stores the sum of the averages of waiting time in the simulations
accWait = 0 # Stores the sum of the time between stoping the arrivals and ending the proccess
lastClient = None # Last client of this simulation
accMax = 0 # Stores the sum of the maximun waiting time in all the process in the simulation
maxDuration = 0 # Maximun duration seen in this simulation
          
def client(env, name, servers, dist_servers, serviceTime):
    t = env.now
    
    for i in range(len(servers)):
        with servers[i].request() as req:
            yield req # wait for the server to accept the client
            yield env.timeout(dist_servers[i]()) # wait until the server finishs with the client

    # Finished with all n servers
            
    global totalTime
    global maxDuration
    global lastClient
    global accMax

    totalTime += (env.now - t) 
    
    if(env.now - t > maxDuration):
            maxDuration = env.now - t
    
    if(name == lastClient):
        global accWait
        accWait += (env.now - serviceTime) 
        accMax += maxDuration
        maxDuration = 0
    
def setup(env, num_servers, dist_arrival, dist_servers, time):
    # Setups the simulation and creates the client´s process
    servers = [simpy.Resource(env) for _ in range(num_servers)]
    
    # Create clients
    i = 0
    while (env.now < time):
        yield env.timeout(dist_arrival(env.now, time)) # wait until new client arrives
        newClient = client(env, f'Client {i}', servers, dist_servers, time)
        env.process(newClient) # process new client
        i += 1

    global lastClient
    lastClient = f'Client {i - 1}'
    global avgSum
    avgSum += totalTime/i


# Set number of servers, and time distributions to use in each one
dist_servers = [ lambda: random.expovariate(1.0 / 4),
                 lambda: random.expovariate(1.0 / 2), 
                 lambda: random.expovariate(1.0 / 3)]

def NonHomogenousExponential(intensities, TotalTime, CurrTime):
     max_value = max([value for item, value in intensities])

     cand = random.expovariate(max_value)
     if(TotalTime < CurrTime):
          return cand
     for item, value in intensities:
          if(item >= CurrTime/TotalTime):
               if(random.uniform(0 , 1) < value / max_value):
                    return cand
               return cand + NonHomogenousExponential(intensities, TotalTime, CurrTime + cand)

total_simulations = 1000

for i in range(total_simulations):
    lastClient = None
    totalTime = 0
    
    env = simpy.Environment()
    
    # Setup the environment, and set the time distribution for client arrival
    intensities = [(0.5 , 1.0 / 5) , (1 , 1.0 / 10)]
    env.process(setup(env, len(dist_servers), lambda a, t: NonHomogenousExponential(intensities, a, t), dist_servers, 500))
    
    env.run()  # Run the simulation for a period of time 

print(f"Mean waiting time: {avgSum / total_simulations}")
print(f"Mean death time: {accWait / total_simulations}")
print(f"Mean max waiting time: {accMax / total_simulations}")


Mean waiting time: 23.90629380972774
Mean death time: 31.072935659541322
Mean max waiting time: 58.041239312364716
